In [ ]:
import wandb
import random

# start a new wandb run to track this script
wandb.init(
    # set the wandb project where this run will be logged
    project="my-awesome-project",

    # track hyperparameters and run metadata
    config={
    "learning_rate": 0.02,
    "architecture": "CNN",
    "dataset": "CIFAR-100",
    "epochs": 10,
    }
)

# simulate training
epochs = 10
offset = random.random() / 5
for epoch in range(2, epochs):
    acc = 1 - 2 ** -epoch - random.random() / epoch - offset
    loss = 2 ** -epoch + random.random() / epoch + offset

    # log metrics to wandb
    wandb.log({"acc": acc, "loss": loss})

# [optional] finish the wandb run, necessary in notebooks
wandb.finish()

In [ ]:
import sys
sys.argv

In [ ]:
import argparse
parser = argparse.ArgumentParser()
parser.add_argument('-c', '--config', type=str, default='config/sr3_16_128.json',
                    help='JSON file for configuration')
parser.add_argument('-p', '--phase', type=str, choices=['train', 'val'],
                    help='Run either train(training) or val(generation)', default='train')
parser.add_argument('-gpu', '--gpu_ids', type=str, default=None)
parser.add_argument('-debug', '-d', action='store_true')
parser.add_argument('-enable_wandb', action='store_true')
parser.add_argument('-log_wandb_ckpt', action='store_true')
parser.add_argument('-log_eval', action='store_true')

# parse configs
args = parser.parse_known_args()[0]

In [ ]:
phase = args.phase
opt_path = args.config
gpu_ids = args.gpu_ids
enable_wandb = args.enable_wandb
print(phase, opt_path, gpu_ids, enable_wandb)

In [ ]:
bool = (None==0)
bool

In [8]:
from pathlib import Path
from PIL import Image
import lmdb
import numpy as np

In [7]:
file = Image.open('../datasets/celeba_hq_256/00000.jpg')
file  = np.array(file)
file.nbytes

196608

In [ ]:
file_path = Path('../datasets/celeba_16_128/16_lr')
file = [x for x in file_path.iterdir() if x.is_file()]
file

In [12]:
lmdb_file_path = Path('../datasets/celeba_lmdb_16_128')
env = lmdb.open(str(lmdb_file_path),readonly=True,readahead=False,lock=False)

In [26]:
with env.begin(write=False) as txn:
    img_bytes = txn.get('{}_lr_{}'.format(16,str(5).zfill(5)).encode())
    print(img_bytes)


b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIHDR\x00\x00\x00\x10\x00\x00\x00\x10\x08\x02\x00\x00\x00\x90\x91h6\x00\x00\x02|IDATx\x9cE\x8f\xdbN\x13Q\x18F\xff\xbdg:\x9d\xe9\xb4\xa5\x87\x81\xb6\x94J\x80bl\xaa\xa0Q\x13\x12\x8d\x89z\xe3\x8d\x0f\xe1K\x18\xdf\xca\x87\xf0\xd6D\x03B@\xa0@\xa1\xd0\x03\x9d\xf3q\xcf\xec\xfd{A\x84\xef~}+\x8b|\xfb\xfa\x05@\x00\x00\xa5\x94P\x12\xf8\xe1\xc1\xc1\xc01]\xcf\xf7\xb5\x82Z,\xe9\xbd\xfe\xc6\xf2r#\xcb\xb8\x10\x02\x91\xc8\xf0\x7f\x84\x10\xcbt\x0f\xf7O\xafF\x13MS\x90\xa0\xeb\x06\xb7\xb7\x9ee\xfa\xdd\xcdvws\x95R\x8a\x00\xf2\xdd=!D\x081\xbe\xbaaQD)\xa5@\xd24E\x94\x81\xd2\xf1\xd4\xb2\x1cW\xd3\xd4\xe5\x96\xc1\x05\xca\x88x\x07x^`\x9b\xa6J\xe1\xd3\xce\xb6\x10\xe8\xba~\xc6\xf9\xc8\xb2\xaf\xe7\xeetf\x8f.\'\xedF\x9dp\xf1\x00\xb8\xb6\xabJ\xf4\xf3\xfb7\xfd\x8dM\x10"MY\x1c\x07\x7f\x8e\x8f\xbe\xff\xf8mR\xb0\x1d\x1f\x11\x08\xc0CC\xcay\xa7\xd5\xdcz\xfa\x8c"p\x91\xc9\x12U\x08Y[Y]1\xce\xa6\x8eW\xd2\x0b\x04Q\xdc\x03\x88\x98\xcf\xe7\xf2(\xfd\xdc?\x06\xc4\xad^\xd7\x8f\xb3_{G\tc+K\xad\xc1\xf5

In [27]:
from io import BytesIO
img = Image.open(BytesIO(img_bytes)).convert('RGB')
img

In [39]:
import os
import json
from collections import OrderedDict
from datetime import datetime

def mkdirs(paths):
    if isinstance(paths,str):
        os.makedirs(paths,exist_ok=True)
    else:
        for path in paths:
            os.makedirs(path,exist_ok=True)

def get_timestamp():
    return datetime.now().strftime('%Y%m%d_%H%M%S')

def parse(args):
    phase =args.phase
    config_path = args.config
    gpu_ids = args.gpu_ids
    enable_wandb = args.enable_wandb

    #删除注释并解析json文件
    json_str = ''
    with open(config_path,'r') as f:
        for line in f:
            line = line.split('//')[0] + '\n'
            json_str += line
    opt = json.loads(json_str, object_pairs_hook=OrderedDict) # OrderedDict保证json文件中键的顺序不变

    #设置log文件路径
    if args.debug:
        opt['name'] = 'debug_{}'.format(opt['name'])
    experiments_root = os.path.join('experiments','{}_{}'.format(opt['name'],get_timestamp()))
    opt['path']['experiment_root'] = experiments_root
    for key,path in opt['path'].items():
        if 'resume' not in key and 'expreiments' not in key:
            opt['path'][key] = os.path.join(experiments_root,path)
            mkdirs(opt['path'][key])

    opt['phase'] = phase

    #设置分布式gpu
    if gpu_ids is not None:
        opt['gpu_ids'] = [int(x) for x in gpu_ids.split(',')]
        gpu_list = gpu_ids
    else:
        gpu_list = ','.join([str(x) for x in opt['gpu_ids']])
    os.environ['CUDA_VISIBLE_DEVICES'] = gpu_list
    if len(gpu_list) > 1:
        opt['distributed'] = True
    else:
        opt['distributed'] = False

    #设置wandb
    try:
        wandb_log_checkpoint = args.wandb_log_checkpoint
        opt['wandb']['wandb_log_checkpoint'] = wandb_log_checkpoint
    except:
        pass
    try:
        wandb_log_eval = args.wandb_log_eval
        opt['wandb']['wandb_log_eval'] = wandb_log_eval
    except:
        pass
    try:
        wandb_log_info = args.wandb_log_info
        opt['wandb']['wandb_log_info'] = wandb_log_info
    except:
        pass
    opt['wandb']['enable_wandb'] = enable_wandb

    return opt

    #设置nonedict类，当未匹配到键对时返回None而不是抛出异常
class NoneDict(dict):
    def __missing__(self, key):
        return None
    
#将opt字典中的所有dict转换为Nonedict类
def dict_to_nonedict(opt):
    if isinstance(opt, dict):
        new_opt = dict()
        for key,val in opt.items():
            new_opt[key] = dict_to_nonedict(val)
        return NoneDict(**new_opt)
    elif isinstance(opt,list):
        return [dict_to_nonedict(x) for x in opt]
    else:
        return opt


In [42]:
import argparse
root = Path.cwd().parent/'config'/'sr3_16_128.json'
parser = argparse.ArgumentParser()
parser.add_argument('--config',default=root,type=str,help='config file path')
parser.add_argument('--phase',type = str,default='train',help='train or test')
parser.add_argument('--gpu_ids',type = str,default = None,help='gpu id')
parser.add_argument('--debug',action='store_true')
parser.add_argument('--enable_wandb',action='store_true')
parser.add_argument('-wandb_log_ckpt',action='store_true')
parser.add_argument('--log_eval',action='store_true')

#设置config
args = parser.parse_known_args()
args = args[0]
opt = parse(args)
opt = dict_to_nonedict(opt)

In [17]:
json_str = ''
with open(args.config,'r') as f:
    for line in f:
        line = line.split('//')[0] + '\n'
        json_str += line
opt = json.loads(json_str, object_pairs_hook=OrderedDict) # OrderedDict保证json文件中键的顺序不变

In [ ]:
for key,value in opt['path'].items():
    if 'resume' not in key and 'experiments' not in key:
        print(key,value)
        print(type(key),type(value))
        print(type('resume'))

log logs_sr3_16_128
<class 'str'> <class 'str'>
<class 'str'>
tb_logger tb_logs_sr3_16_128
<class 'str'> <class 'str'>
<class 'str'>
result result
<class 'str'> <class 'str'>
<class 'str'>
checkpoint checkpoint
<class 'str'> <class 'str'>
<class 'str'>


In [6]:
type(shape[1:])
a,b,c = *shape[1:]

SyntaxError: can't use starred expression here (1985169556.py, line 2)

In [10]:
shape = (64,3,128,128)
(64,*([1]*len(shape[:-1])))

(64, 1, 1, 1)